## IMAGENET 影像辨識比賽
* 約100萬張影像，共1000個類別
* 有許多深度學習框架也納入這些模型，提供我們方便直接使用: [[Tensorflow]](https://www.tensorflow.org/api_docs/python/tf/keras/applications) [[Keras]](https://keras.io/api/applications/)[[Pytorch]](https://pytorch.org/hub/research-models)

In [ ]:
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.applications.mobilenet import MobileNet
#from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np

In [ ]:
img_path = "elephant.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, 0)

In [ ]:
model = ResNet50(weights="imagenet")
#model = MobileNet(weights="imagenet")

In [ ]:
model.summary()

* decode_predictions(preds, top=n) 可以預測前top=n的數量的預測值，預設為n=5

In [ ]:
preds = model.predict(x)
print("Predicted:", decode_predictions(preds, top=3)[0])

## Grad-CAM CNN視覺化 [[論文]](https://arxiv.org/abs/1610.02391)

* Numpy中的maximum：比較兩個陣列對應元素並取最大值 [[連結]](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html)
* Keras中的backend.mean：張量在某一指定軸的平均值 [[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/backend)


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
conv_layer = model.get_layer("conv4_block6_2_conv")                               # 獲取預提取的卷積層
heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])   # 建一「輸入層」至「預提取的卷積層為輸出層」的模型

with tf.GradientTape() as gtape:
    conv_output, predictions = heatmap_model(x)
    prob = predictions[:, np.argmax(predictions[0])]  # 取機率最大的類別之機率
    grads = gtape.gradient(prob, conv_output)         # 類別與卷積層的梯度
    pooled_grads = K.mean(grads, axis=(0, 1, 2))      # 每層權重平

heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)  #權重與特徵圖相乘，並求和平均
heatmap = np.maximum(heatmap, 0)    # 比較兩個陣列對應元素取最大值。若一陣列是給0，則代表另一陣列的值若是負數，則變為0。
heatmap /=  np.max(heatmap)         # 正規化
heatmap = heatmap[0]

plt.imshow(heatmap)
plt.show()

### 熱圖與原圖結合
* OpenCV中的applyColorMap [[連結]](https://docs.opencv.org/4.x/d3/d50/group__imgproc__colormap.html)
* OpenCV中的addWeighted [[連結]](https://docs.opencv.org/3.4/d5/dc4/tutorial_adding_images.html)

In [ ]:
import cv2

def add_heatmap(heatmap, image, alpha=0.8, colormap=cv2.COLORMAP_VIRIDIS):
    heatmap = cv2.applyColorMap(heatmap, colormap)                 # 偽顏色映射
    output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)  # 兩個影像疊加
    return output

In [ ]:
img = cv2.imread(img_path)

heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0])) # 將熱圖的影像更改大小與原圖相同
heatmap = np.uint8(255*heatmap)                             # 將正規化的數值轉為原來數值，即乘以255，並轉為整數型別

output = add_heatmap(heatmap, img, alpha=0.5)
output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

plt.imshow(output)
plt.show()

** *

# 黃豆影像資料集
* 清華大學與合作農場所搜集的黃豆影像
* 訓練集有19,198張影像，測試集有4,785張影像

## **作業**

* 以黃豆資料集訓練任意兩個經典的卷積神經網路模型 (40%)
* 比較兩個模型的測試集之混淆矩陣、準確度、敏感度、特異度與ROC曲線 (20%，各4%)
* 以準確度最高的模型使用Grad-CAM呈現視覺化熱圖，測試一張壞掉的黃豆，具有顯著的壞掉特徵 (40%) 

### ※ 相關黃豆論文[[連結]](https://www.sciencedirect.com/science/article/abs/pii/S0168169921002477)
![](https://ars.els-cdn.com/content/image/1-s2.0-S0168169921002477-gr12.jpg)